In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [51]:
var('xx')
Q.<alpha> = NumberField(xx^2+1)

In [52]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [53]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [54]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [55]:
line = Line([y, z])

In [56]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [57]:
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [58]:
%time adm_SE1 = SE1.find_admissible_projectivities()

CPU times: user 12.4 s, sys: 454 ms, total: 12.9 s
Wall time: 21.3 s


In [59]:
len(adm_SE1)

576

In [60]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)

CPU times: user 3.17 s, sys: 301 ms, total: 3.47 s
Wall time: 21.7 s


In [61]:
len(simm_SE1)

2

In [62]:
G_SE1 = Group(simm_SE1)

In [63]:
G_SE1.is_group()

True

In [64]:
G_SE1.get_order()

2

In [65]:
G_SE1.is_abelian()

True

In [66]:
G_SE1.get_divisors_of_order()

[1, 2]

In [67]:
G_SE1.get_order_of_elements()

[1, 2]

In [68]:
G_SE1.apply_to(SE1.eckardt_points[0])

[(0, 0, 0, 1), (0, 0, 0, 1)]

In [69]:
adm_perm = SE1.find_admissible_permutations()
keys = list(SE1.cl_lines.keys())

In [70]:
len(adm_perm)

1152

In [71]:
def apply_proj_to_lines(proj, lines):
    new_indices = []
    for i in range(len(lines)):
        new_indices.append(lines.index(lines[i].apply_proj(proj))+1)
    return new_indices

In [72]:
M2 = [mat for mat in simm_SE1 if mat != matrix.identity(4)][0]
M2_perm = Permutation(apply_proj_to_lines(M2, list(SE1.cl_lines.values()))).to_permutation_group_element()
print(M2_perm)

(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23)


In [73]:
perms_lines = []
for perm in adm_perm:
    perms_lines.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

In [74]:
Gr = PermutationGroup(perms_lines)

In [75]:
Gr.order()

1152

In [76]:
M2_perm.order()

2

In [77]:
el_order_4 = []
for el in Gr.list():
    iden = Gr.identity()
    if el^2 == M2_perm and el^3 != iden and el^4 == iden:
        el_order_4.append(el)

In [78]:
len(el_order_4)

12

In [79]:
def from_perm_to_labels(perm):
    keys = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'F12', 'F13', 'F14', 'F15', 'F16', 'F23', 'F24', 'F25', 'F26', 'F34', 'F35', 'F36', 'F45', 'F46', 'F56']
    return [keys[perm.dict()[key]-1] for key in perm.dict()]

In [80]:
labels_el_order_4 = [from_perm_to_labels(el) for el in el_order_4]

In [81]:
projs = SE1.find_admissible_projectivities(labels_el_order_4)

In [82]:
def find_conditions_for_subfamilies(cubic, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    sing_cubics_factored = cubic.sing_cubic.factor()
    for M in [proj for proj in projectivities if proj not in simmetries]:
        print(projectivities.index(M))
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), sing_cubics_factored)    
        minor = matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)
        minor = list(set([remove_sing_factors(el, sing_cubics_factored) for el in minor if el !=0]))
        ideale = cubic.P.ideal(minor)
        new_ideals = get_valid_ideals(ideale, cubic, sing_cubics_factored)
        for ideal in new_ideals:            
            prim_deco2 = (ideale + ideal).radical().primary_decomposition()
            for ideal2 in prim_deco2:
                if is_ideal_valid(ideal2, cubic, sing_cubics_factored):
                    conditions.append(ideal2.gens())                        
    return list(set(conditions))


def get_valid_ideals(ideale, cubic, sing_cubics_factored):
    valid_ideals = []
    vrs = [c,d,e,f]
    for vr in vrs:
        valid_ideals_var = []
        del_ideale = ideale.elimination_ideal([vr])
        prim_deco_del = del_ideale.radical().primary_decomposition()
        for ideal in prim_deco_del:
            if is_ideal_valid(ideal, cubic, sing_cubics_factored):
                valid_ideals_var.append(ideal)
        if valid_ideals_var == []:
            return []
        else:
            valid_ideals.append(valid_ideals_var)
    return [id1+id2+id3+id4 for id1 in valid_ideals[0] for id2 in valid_ideals[1] for id3 in valid_ideals[2] for id4 in valid_ideals[3]]           

In [83]:
%time conds = find_conditions_for_subfamilies(SE1, projs, simm_SE1)

0
1
2
3
4
5
6
7
8
9
10
11
CPU times: user 4min 34s, sys: 4.04 s, total: 4min 38s
Wall time: 4min 41s


In [88]:
sost = solve_linear_system([f, 25*d + (-2*alpha - 11)*e, 5*c + (2*alpha + 1)*e], [d,e,f], [c])
sost

[(-10*alpha - 55)*c, -125*c, 0, (50*alpha + 25)*c]

In [89]:
var = [c,d,e,f]

In [91]:
nc = SE1.subs({var[i]:sost[i] for i in range(4)})

In [96]:
nc_simm = nc.find_simmetries(nc.find_admissible_projectivities())

In [101]:
[pl.conditions for pl in nc.tritangent_planes]

[(-34715000*alpha - 44370000),
 (-300*alpha - 7900),
 0,
 (-4100*alpha - 3800),
 (7900*alpha - 300),
 (248512750000*alpha - 137211125000),
 (-6600*alpha - 17550),
 (-150*alpha - 3950),
 (36250*alpha - 347500),
 (-860000*alpha - 511250),
 (42505000*alpha + 43777500),
 (-64010000*alpha - 120492500),
 (-4550*alpha - 15650),
 (-395250*alpha - 252000),
 (-596500*alpha - 343250),
 (-83825000*alpha - 71975000),
 (248512750000*alpha - 137211125000),
 (-42505000*alpha - 43777500),
 (6908525000*alpha + 179700000),
 (3802725000*alpha - 994387500),
 (-6908525000*alpha - 179700000),
 (-603555000000*alpha + 22400625000),
 (40135000*alpha + 12617500),
 (1398500*alpha + 1410500),
 (-30550000*alpha - 10212500),
 (3802725000*alpha - 994387500),
 (226378750000*alpha - 126909062500),
 (5672500*alpha + 23073750),
 (502250*alpha + 1246750),
 (10456250*alpha + 34462500),
 (-64010000*alpha - 120492500),
 (-226378750000*alpha + 126909062500),
 (603555000000*alpha - 22400625000),
 (6600*alpha + 17550),
 (-56725

In [21]:
with open('conditions.pickle', 'rb') as ff:
    conditions = pickle.load(ff)